In [59]:
from bs4 import BeautifulSoup
import requests
import os
import lxml


In [60]:
print(os.getcwd())

C:\Users\Mike Zhong\Dropbox\dev\fantasy_analytics


In [61]:
my_url = "http://www.footballdb.com/fantasy-football/index.html?pos=QB%2CRB%2CWR%2CTE&yr=2016&wk=1&rules=1"

html_page = requests.get(my_url)

if html_page.status_code != 200:
    print("Non-200 response code returned, exiting...")
    exit(1)

print(type(html_page))
print(html_page)
print(html_page.text[:400])

with open("./html/sample_html.html", 'w+') as fp:
    fp.write(html_page.text)

<class 'requests.models.Response'>
<Response [200]>
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="viewport" content="initial-scale=1.0, width=device-width" />
<title>2016 Fantasy Football Statistics | The Football Database</title>
<meta name="description" content="View week 1 QB,RB,WR,TE fantasy football stats and statistics for the 2016 NFL season. Included are 


In [62]:
soup = BeautifulSoup(html_page.text, 'lxml')

print(type(soup))

all_tables = soup.find_all("table")
print(len(all_tables), type(all_tables))

<class 'bs4.BeautifulSoup'>
7 <class 'bs4.element.ResultSet'>


In [73]:
table = all_tables[0]
trs = table.find_all("tr")
print(len(trs), type(trs))

tr = trs[1]
ths = tr.find_all("th")
print(len(ths), type(ths))

102 <class 'bs4.element.ResultSet'>
19 <class 'bs4.element.ResultSet'>


In [98]:
rows = []

headers = trs[1].text.strip().split('\n')
ncols = len(headers)
print(len(headers), type(headers))

tds = trs[2].find_all('td')
print(len(tds))

19 <class 'list'>
19


In [120]:
import pandas as pd
for tr in trs[2:]:
    
    row = []
    
    for td in tr.find_all('td'):
        
        if len(td) > 0:    
            row.append(td.text.strip().split('.')[0])
        
    rows.append(row)
    
my_table = pd.DataFrame(rows, columns=headers)

print(my_table.size)
print(my_table.shape)
some_row = my_table[1:3]

print(some_row, type(some_row))
            
    

24700
(1300, 19)
                Player   Opp Pts* Att Cmp  Yds TD Int 2Pt Att Yds TD 2Pt Rec  \
1      Drew Brees, NOD   Oak   39  42  28  423  4   0   0   2   5  0   0   0   
2  Jameis Winston, TBJ  @Atl   33  32  23  281  4   1   0   4   3  0   0   0   

  Yds TD 2Pt FL TD  
1   0  0   0  1  0  
2   0  0   0  0  0   <class 'pandas.core.frame.DataFrame'>


In [125]:
my_table.Player

0                  Andrew Luck, IndA
1                    Drew Brees, NOD
2                Jameis Winston, TBJ
3             Matthew Stafford, DetM
4                    Alex Smith, KCA
5            DeAngelo Williams, PitD
6                    Matt Ryan, AtlM
7           Ben Roethlisberger, PitB
8                 Brandin Cooks, NOB
9                 Aaron Rodgers, GBA
10                 Spencer Ware, KCS
11                                 C
12                 Eli Manning, NYGE
13                                 A
14               Antonio Brown, PitA
15                Carson Wentz, PhiC
16                 Willie Snead, NOW
17                  Derek Carr, OakD
18                Theo Riddick, DetT
19               Carson Palmer, AriC
20                  Cam Newton, CarC
21              Brock Osweiler, HouB
22              Marcus Mariota, TenM
23                  Carlos Hyde, SFC
24            Larry Fitzgerald, AriL
25              DeMarco Murray, TenD
26               David Johnson, AriD
2